---
# Introduction <a name="introduction"></a>

This notebook provides a comprehensive guide for managing prompt template assets developed using Language Models (LLMs) across various platforms. It covers:

- **LLMs on Third-Party Platforms**: Including AWS Bedrock , Azure and more.
- **Standard Prompts**: Created directly within the `watsonx.ai` platform.

The notebook offers insights into the management and utilization of these prompt templates to optimize their use across different environments.


For detailed documentation on these features, please refer to the [IBM AI Factsheet documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/index.html).


**Required Services:**
- `watsonx.governance`

**Required Packages:**
- **IBM Facts Client Python SDK (>=1.0.80)**




In [ ]:
!pip install ibm-aigov-facts-client
!pip install -U python-dotenv --quiet

In [7]:
import warnings
#import wget
import shutil
import os 
from dotenv import load_dotenv
from IPython.core.display import display, Markdown
from ibm_aigov_facts_client import AIGovFactsClient
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig,DeploymentDetails,TrainingDataReference,ExternalModelSchemas,ModelDetails,DetachedPromptTemplate,PromptTemplate

warnings.filterwarnings('ignore')
load_dotenv()

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

- This sample will use IBM Cloud by default. If you prefer to use Cloud pak for data, set `use_cp4d=True`
- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [9]:
use_software=False
run_cleanup_at_end=True

- Model container type can be `space` or `project`

In [10]:
container_type="project"
container_id=os.getenv("CONTAINER_ID", "<if you can't use .env you can provide your value here>") # Project_id where the model has to store 

---
## Authentication Setup<a name="setup"></a>

### IBM Cloud  <a name="IBM-Cloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [11]:
if not use_software:
    API_KEY=os.getenv("CLOUD_API_KEY", "<if you can't use .env you can provide your value here>")

### IBM watsonx.goverance software<a name="Watsonx.Gov-Platform"></a>

- Service url is the watsonx.goverance software platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate

In [12]:
if use_software:
    creds=CloudPakforDataConfig(service_url=os.getenv("CPD_SERVICE_URL", "<if you can't use .env you can provide your value here>"),
        username=os.getenv("CPD_USERNAME", "<if you can't use .env you can provide your value here>"),
        password=os.getenv("CPD_PASSWORD", "<if you can't use .env you can provide your value here>"))


## Client Initialization
- Container type would be either `space` or `project`. To use get/set environment utilities, model asset should be stored in Space.
- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client





In [19]:
if use_software:
   facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,container_type=container_type,container_id=container_id,disable_tracing=True)
else: 
   facts_client = AIGovFactsClient(api_key=API_KEY,container_type=container_type,container_id=container_id,disable_tracing=True)

In [20]:
facts_client.version 

'1.0.79'

---
## Creating Detached Prompt <a name="Creating-a-Detached-Prompt"></a>

In creating a Detached Prompt, there are two primary classes to consider:

- **DetachedPromptTemplate**
- **PromptTemplate** (optional)

### Detached Prompt <a name="Detached-Template"></a>

A detached prompt template is an asset used to evaluate a prompt template for a language prompt hosted by third-party providers like Google Vertex AI, Azure OpenAI, or AWS Bedrock. The output is generated on the remote model, while evaluation is conducted using `watsonx.governance` metrics. You can also track the detached deployment and prompt template as part of your AI governance solution.

You add parts of your prompt into the appropriate fields by adding it to `DetachedPromptTemplate`:

- **prompt_id**: Identifier for the prompt, applicable as supported by the provider.
- **model_id**: Unique identifier for the provisioned model, often in ARN or equivalent format.
- **model_provider**: The provider of the model.
- **model_name**: The name of the model being used.
- **model_url**: Link to the model's documentation or additional resources.
- **prompt_url**: URL for the prompt, if supported by the provider.
- **prompt_additional_info**: Dictionary for any supplementary information relevant to the prompt, such as configuration settings.


**Note:** When providing Detached information:
- **Model Information:** You need to include both `model_name` and `model_url`. If you leave one out, you should leave both out.
- **Prompt Information:** The same applies to `prompt_url` and `prompt_additional_info`.


In [21]:
# define parameters for DetachedPromptTemplate

detached_information =DetachedPromptTemplate(
    prompt_id="<prompt id when supported by provider>", 
    model_id="arn:aws:bedrock:us-east1:123456789012:provisioned-model/anthropic.claude-v2",
    model_provider="AWS Bedrock",
    model_name="Anthropic Claude 2.0",
    model_url="https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html",
    prompt_url="<prompt url when supported by provider>",
    prompt_additional_info={"AWS Region": "us-east1"}
)

### Prompt Template <a name="Prompt-Template"></a>
The `PromptTemplate` class is designed to create and manage templates for prompts.It includes a variety of parameters to configure the prompt template comprehensively. 

You add parts of your prompt into the appropriate fields by adding it to `PromptTemplate`:

- **Input**: The text or data you provide as the starting point for the prompt. This is the information the model will work with.
  
- **Prompt Variables**: A prompt variable is a placeholder keyword that you include in the static text of your prompt at creation time and replace with text dynamically at run time.

- **Model Parameters**: These settings control how the model generates responses. For example:

  - `decoding_method`: "greedy"
  - `max_new_tokens`: 2034
  - `min_new_tokens`: 0
  - `random_seed`: 0
  - `top_k`: 0
  - `top_p`: 0

In [22]:
model_parameter={"decoding_method":"greedy",
                 "max_new_tokens":2034,
                 "min_new_tokens":0,
                 "random_seed":0,
                 "top_k":0,
                 "top_p":0
                }

In [23]:
prompt_template = PromptTemplate(
                                 prompt_variables= {"text": "value" },
                                 input="Input text to be given ",

                                 model_parameters=model_parameter
                                 
                                )

### Setting Up a Detached Prompt

The `facts_client.assets.create_detached_prompt` function is used to create a detached prompt.

#### Parameter Descriptions for Detached Prompt

- **task_id**: Specifies the task the prompt is designed for:
  - **question_answering**: Generates answers to questions.
  - **summarization**: Condenses content into a brief summary.
  - **retrieval_augmented_generation**: Combines information retrieval with generation.
  - **classification**: Categorizes text.
  - **generation**: Creates new content.
  - **code_generation_and_conversion**: Generates or converts code.
  - **extraction**: Pulls out specific information.
  - **translation**: Converts text from one language to another.

- **model_id**: Specifies the model used to create the detached prompt.

- **name**: A label for easy identification of the prompt.

- **description**: A brief explanation of the prompt's purpose, used for documentation.

- **detached_information**: Contains the prompt template detailing how the detached prompt should be defined.

- **prompt_details**: Further outlines specifics for defining the prompt.


In [24]:
detached_prompt=facts_client.assets.create_detached_prompt(model_id="anthropic.claude-v2",task_id="classification", name="External prompt sample (model AWS Bedrock Anthropic)", description="My first detached prompt",prompt_details=prompt_template, detached_information=detached_information)

2024/08/09 21:14:46 INFO : ------------------------------ Detached Prompt Creation Started ------------------------------
2024/08/09 21:14:48 INFO : The detached prompt with ID 218352c9-600e-4ca9-8725-bb51c1a9bdf2 was created successfully in container_id 544caa9f-82a5-4f5c-900e-db36097cc71e.


In [25]:
detached_prompt.get_info(verbose=True)


2024/08/09 21:14:51 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/218352c9-600e-4ca9-8725-bb51c1a9bdf2/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:14:51 INFO : 
Prompt Details:
  id: 218352c9-600e-4ca9-8725-bb51c1a9bdf2
  name: External prompt sample (model AWS Bedrock Anthropic)
  input mode: detached
  model_id: anthropic.claude-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e
  description: My first detached prompt
  created By: IBMid-6910001S2E
  model_version: {}
  prompt_variables: {'text': {'default_value': 'value'}}
  task_id: ['classification']
  instruction: 
  input_prefix: 
  output_prefix: 
  structured_examples:
  input:
    ['Input text to be given ', '']

  model_parameters:
     decoding_method: greedy
     max_new_tokens: 2034
     min_new_tokens: 0
     random_seed: 0
     stop_sequences: []
     temperature: 
     top_k: 0
     top_p: 0
     repetition_penalty:

#### Fetch the asset_id of the prompt

In [27]:
#extract the assest_id from above 

asset_id=detached_prompt.get_asset_id()
print(asset_id)


218352c9-600e-4ca9-8725-bb51c1a9bdf2


---

## Creating Regular Prompts <a name="Creating-a-Detached-Prompt"></a>

In crafting regular prompts, there are primarily two types to consider:

- **Structured**
- **Freeform**

To create these prompts, a fundamental base class is required:
- **PromptTemplate**



### Structured Prompt
Structured mode is designed to help new users create effective prompts. Text from the fields is sent to the model in a template format.

You add parts of your prompt into the appropriate fields by adding it to `PromptTemplate`:

- **Input**: The text or data you provide as the starting point for the prompt. This is the information the model will work with.
  
- **Prompt Variables**: These are placeholders in the template that will be replaced with specific input values when the prompt is used.

- **Instruction**: Add an instruction if it makes sense for your use case. An instruction is an imperative statement, such as "Summarize the following article."

- **Examples**: Add one or more pairs of examples that contain the input and the corresponding output that you want. Providing a few example input-and-output pairs in your prompt is called few-shot prompting.

- **Input and Output Prefix**: If you need a specific prefix to the input or the output, you can use the input and output prefix. For example, you might replace the default labels with custom labels that were used in training data when a foundation model was prompt-tuned.

- **Model Parameters**: These settings control how the model generates responses. For example:

  - `decoding_method`: "greedy"
  - `max_new_tokens`: 2034
  - `min_new_tokens`: 0
  - `random_seed`: 0
  - `top_k`: 0
  - `top_p`: 0


In [28]:
model_parameter={"decoding_method":"greedy",
                 "max_new_tokens":2034,
                 "min_new_tokens":0,
                 "random_seed":0,
                 "top_k":0,
                 "top_p":0
                }

In [29]:
prompt_template = PromptTemplate(model_version={"number": "2.0.0-rc.7", "tag": "tag", "description": "Description of the model version"},
                             input="Input text to be given",
                             prompt_variables= {"text": "value"},
                             prompt_instruction="Your prompt instruction",
                             input_prefix="Your input prefix",
                             output_prefix="Your output prefix",
                             structured_examples={"What is the capital of France{text}?": "The capital of France is Paris{text}.",
                                        "Who wrote '1984{text}'?": "George Orwell wrote '1984'{text}."},
                            model_parameters=model_parameter
)

### Setting Up a Structured Prompt

The `facts_client.assets.create_prompt` function is used to create a structured prompt.

#### Parameter Descriptions for Structured Prompt

- **input_mode**: Specifies the format for creating the prompt:
  - **structured**: Utilizes a template for effective prompt creation.
  - **freeform**: Allows for more creative and flexible prompt creation.

- **task_id**: Defines the specific task the prompt is designed for:
  - **question_answering**: Generates answers to questions.
  - **summarization**: Condenses content into a brief summary.
  - **retrieval_augmented_generation**: Merges retrieval of information with generative capabilities.
  - **classification**: Categorizes or classifies text.
  - **generation**: Creates new content.
  - **code_generation_and_conversion**: Generates or converts code.
  - **extraction**: Pulls out specific information from text.
  - **translation**: Converts text from one language to another.

- **model_id**: Specifies the model used for generating responses. Example:
  - **ibm/granite-13b-chat-v2**: Supported by watsonx.ai. 
  Users can choose from recently used models or other available foundation models, including tuned models from Tuning Studio.

- **name**: A label for the prompt that helps identify it easily.

- **description**: A brief explanation of what the prompt is about, used for documentation

- **prompt_details**: Contains the prompt template outlining how the prompt should be structured or specifics it should included


In [30]:
structured_prompt =facts_client.assets.create_prompt(input_mode="structured",
                                                      name=" structured prompt sample",
                                                      task_id="summarization",
                                                      model_id="ibm/granite-13b-chat-v2", 
                                                      description="My First structured prompt",
                                                      prompt_details=prompt_template,
                                                      )

2024/08/09 21:15:08 INFO : ------------------------------ Structured Prompt Creation Started ------------------------------
2024/08/09 21:15:10 INFO : The structured prompt with ID ff222d89-69e3-4000-be83-a851e485a826 was created successfully in container_id 544caa9f-82a5-4f5c-900e-db36097cc71e.


In [31]:
structured_prompt.get_info()

2024/08/09 21:15:21 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/ff222d89-69e3-4000-be83-a851e485a826/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:15:21 INFO : 
Prompt Details:
  id: ff222d89-69e3-4000-be83-a851e485a826
  name:  structured prompt sample
  input mode: structured
  model_id: ibm/granite-13b-chat-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e



#### Fetch the asset_id of the prompt

In [32]:
#extract the assest_id from above 

asset_id=structured_prompt.get_asset_id()
print(asset_id)


ff222d89-69e3-4000-be83-a851e485a826


---
### Freeform Prompt
Freeform mode is designed in which you add your prompt in plain text. Your prompt text is sent to the model exactly as you typed it.
it is a good choice when you want to submit structured input and know how to format the prompt

You add parts of your prompt into the appropriate fields by adding it to `PromptTemplate`:

- **Input**: The text or data you provide as the starting point for the prompt. This is the information the model will work with.
  
- **Prompt Variables**: A prompt variable is a placeholder keyword that you include in the static text of your prompt at creation time and replace with text dynamically at run time.

- **Model Parameters**: These settings control how the model generates responses. For example:

  - `decoding_method`: "greedy"
  - `max_new_tokens`: 2034
  - `min_new_tokens`: 0
  - `random_seed`: 0
  - `top_k`: 0
  - `top_p`: 0


In [33]:
model_parameter={"decoding_method":"greedy",
                 "max_new_tokens":2034,
                 "min_new_tokens":0,
                 "random_seed":0,
                 "top_k":0,
                 "top_p":0
                }

In [34]:
prompt_template = PromptTemplate(
                             input="Input text to be given",
                             prompt_variables= {"text": "value"},

                            model_parameters=model_parameter
                            )

### Setting Up a Freeform Prompt

The `facts_client.assets.create_prompt` function is used to create a Freeform prompt.

#### Parameter Descriptions for Freeflow Prompt

- **input_mode**: Specifies the format for creating the prompt
  - **freeform**:  Utilizes Freeform template for effective prompt creation

- **task_id**: Defines the specific task the prompt is designed for:
  - **question_answering**: Generates answers to questions.
  - **summarization**: Condenses content into a brief summary.
  - **retrieval_augmented_generation**: Merges retrieval of information with generative capabilities.
  - **classification**: Categorizes or classifies text.
  - **generation**: Creates new content.
  - **code_generation_and_conversion**: Generates or converts code.
  - **extraction**: Pulls out specific information from text.
  - **translation**: Converts text from one language to another.

- **model_id**: Specifies the model used for generating responses. Example:
  - **ibm/granite-13b-chat-v2**: Supported by watsonx.ai. 
  Users can choose from recently used models or other available foundation models, including tuned models from Tuning Studio.

- **name**: A label for the prompt that helps identify it easily.

- **description**: A brief explanation of what the prompt is about, used for documentation

- **prompt_details**: Contains the prompt template outlining how the prompt should be structured or specifics it should included


In [35]:
freeform_prompt = facts_client.assets.create_prompt(input_mode="freeform",
                                                    name="Freeflow prompt sample",
                                                    task_id="summarization",
                                                    model_id="ibm/granite-13b-chat-v2", 
                                                    description="My First Freeflow prompt",
                                                    prompt_details=prompt_template,
                                                    )

2024/08/09 21:15:28 INFO : ------------------------------ Freeform Prompt Creation Started ------------------------------
2024/08/09 21:15:29 INFO : The freeform prompt with ID f4ada477-9644-486e-9b07-22ca1584dd07 was created successfully in container_id 544caa9f-82a5-4f5c-900e-db36097cc71e.


In [36]:
freeform_prompt.get_info()

2024/08/09 21:15:31 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/f4ada477-9644-486e-9b07-22ca1584dd07/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:15:31 INFO : 
Prompt Details:
  id: f4ada477-9644-486e-9b07-22ca1584dd07
  name: Freeflow prompt sample
  input mode: freeform
  model_id: ibm/granite-13b-chat-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e



#### Fetch the asset_id of the prompt

In [37]:
#extract the assest_id from above 

asset_id=freeform_prompt.get_asset_id()
print(asset_id)


f4ada477-9644-486e-9b07-22ca1584dd07


---
## Prompt Methods

These methods allow you to modify and view created detached prompts, enabling effective management and optimization of prompt templates for various tasks and use cases.


### Determine Detached Status

Check if the fetched prompt is in Detache mode.

In [38]:
detached_prompt.is_detached()

True

In [39]:
structured_prompt.is_detached()

False

In [40]:
freeform_prompt.is_detached()

False

### Retrieve Prompt Variables
Get the prompt variables of the fetched prompt.
This method retrieves the prompt variables from the fetched prompt.


In [41]:
detached_prompt.get_prompt_variables()

2024/08/09 21:15:36 INFO : Successfully retrieved prompt variables: {'text': {'default_value': 'value'}}


In [42]:
structured_prompt.get_prompt_variables()

2024/08/09 21:15:37 INFO : Successfully retrieved prompt variables: {'text': {'default_value': 'value'}}


In [43]:
freeform_prompt.get_prompt_variables()

2024/08/09 21:15:38 INFO : Successfully retrieved prompt variables: {'text': {'default_value': 'value'}}


### Update prompt variables

This method takes a dictionary of prompt variables and updates the prompt variables of the prompt.

In [44]:
new_prompt_variables = {
    "text_1": "new_text1",
    "user_1": "new_user1"
  
}

In [45]:
detached_prompt.update_prompt_variables(prompt_variables=new_prompt_variables)


2024/08/09 21:15:43 INFO : Prompt variables updated successfully.


In [46]:
structured_prompt.update_prompt_variables(prompt_variables=new_prompt_variables)

2024/08/09 21:15:45 INFO : Prompt variables updated successfully.


In [47]:
freeform_prompt.update_prompt_variables(prompt_variables=new_prompt_variables)

2024/08/09 21:15:46 INFO : Prompt variables updated successfully.


In [48]:
# to see the new changes

detached_prompt.get_info(verbose=True)

structured_prompt.get_info(verbose=True)

freeform_prompt.get_info(verbose=True)

2024/08/09 21:15:48 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/218352c9-600e-4ca9-8725-bb51c1a9bdf2/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:15:48 INFO : 
Prompt Details:
  id: 218352c9-600e-4ca9-8725-bb51c1a9bdf2
  name: External prompt sample (model AWS Bedrock Anthropic)
  input mode: detached
  model_id: anthropic.claude-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e
  description: My first detached prompt
  created By: IBMid-6910001S2E
  model_version: {}
  prompt_variables: {'text': {'default_value': 'value'}, 'text_1': {'default_value': 'new_text1'}, 'user_1': {'default_value': 'new_user1'}}
  task_id: ['classification']
  instruction: 
  input_prefix: 
  output_prefix: 
  structured_examples:
  input:
    ['Input text to be given ', '']

  model_parameters:
     decoding_method: greedy
     max_new_tokens: 2034
     min_new_tokens: 0
     random_seed: 0
     stop_s

### Update Prompt Name

This method allows the setting or updating of the prompt ‘name’ to a new value.

In [51]:
detached_prompt.update_prompt_name(name="detached-NewName")


2024/08/09 21:16:20 INFO : Successfully updated prompt name: 'detached-NewName'.


In [52]:
structured_prompt.update_prompt_name(name="structured-NewName")


2024/08/09 21:16:21 INFO : Successfully updated prompt name: 'structured-NewName'.


In [53]:
freeform_prompt.update_prompt_name(name="freeform-NewName")


2024/08/09 21:16:22 INFO : Successfully updated prompt name: 'freeform-NewName'.


In [54]:
# to see the new changes

detached_prompt.get_info(verbose=True)

structured_prompt.get_info(verbose=True)

freeform_prompt.get_info(verbose=True)

2024/08/09 21:16:23 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/218352c9-600e-4ca9-8725-bb51c1a9bdf2/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:16:23 INFO : 
Prompt Details:
  id: 218352c9-600e-4ca9-8725-bb51c1a9bdf2
  name: detached-NewName
  input mode: detached
  model_id: anthropic.claude-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e
  description: My first detached prompt
  created By: IBMid-6910001S2E
  model_version: {}
  prompt_variables: {'text': {'default_value': 'value'}, 'text_1': {'default_value': 'new_text1'}, 'user_1': {'default_value': 'new_user1'}}
  task_id: ['classification']
  instruction: 
  input_prefix: 
  output_prefix: 
  structured_examples:
  input:
    ['Input text to be given ', '']

  model_parameters:
     decoding_method: greedy
     max_new_tokens: 2034
     min_new_tokens: 0
     random_seed: 0
     stop_sequences: []
     temperature: 
    

### Update Prompt Description

This method allows for the setting or updating of prompt description to a new value.

In [55]:
detached_prompt.update_prompt_description(prompt_description="detached-description")

2024/08/09 21:16:54 INFO : Successfully updated prompt description: 'detached-description'.


In [56]:
structured_prompt.update_prompt_description(prompt_description="structured-description")

2024/08/09 21:16:55 INFO : Successfully updated prompt description: 'structured-description'.


In [57]:
freeform_prompt.update_prompt_description(prompt_description="freeform-description")

2024/08/09 21:16:56 INFO : Successfully updated prompt description: 'freeform-description'.


In [58]:
# to see the new changes

detached_prompt.get_info()

structured_prompt.get_info()

freeform_prompt.get_info()

2024/08/09 21:16:57 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/218352c9-600e-4ca9-8725-bb51c1a9bdf2/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:16:57 INFO : 
Prompt Details:
  id: 218352c9-600e-4ca9-8725-bb51c1a9bdf2
  name: detached-NewName
  input mode: detached
  model_id: anthropic.claude-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e

detached information:
  prompt_id: <prompt id when supported by provider>
  model_id: arn:aws:bedrock:us-east1:123456789012:provisioned-model/anthropic.claude-v2
  model_provider: AWS Bedrock

2024/08/09 21:16:59 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/ff222d89-69e3-4000-be83-a851e485a826/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:16:59 INFO : 
Prompt Details:
  id: ff222d89-69e3-4000-be83-a851e485a826
  name: structured-NewName
  inpu

### Update Prompt Attribute


This method updates a any attribute of the fetched prompt with the provided key name and value.

NOTE:
`update_prompt_attribute` is only supported for Regular prompts, not for Detached prompt



#### Structured prompts

In [59]:
structured_prompt.update_prompt_attribute(key_name="task_id", value="classification")

2024/08/09 21:17:06 INFO : Prompt attribute 'task_id' updated successfully.


In [60]:
structured_prompt.update_prompt_attribute(key_name="input", value="new input")

2024/08/09 21:17:10 INFO : Prompt attribute 'input' updated successfully.


In [61]:
structured_prompt.update_prompt_attribute(key_name="instruction", value="new prompt_instruction")

2024/08/09 21:17:13 INFO : Prompt attribute 'instruction' updated successfully.


In [62]:
structured_prompt.update_prompt_attribute(key_name="input_prefix", value="new input_prefix")

2024/08/09 21:17:15 INFO : Prompt attribute 'input_prefix' updated successfully.


In [63]:
structured_prompt.update_prompt_attribute(key_name="output_prefix", value="new output_prefix")

2024/08/09 21:17:18 INFO : Prompt attribute 'output_prefix' updated successfully.


In [64]:
new_example =  {
    "What is the capital of India{text}?": "The capital of India is New Delhi{text}."
}

In [65]:
structured_prompt.update_prompt_attribute(key_name="structured_examples",value=new_example)

2024/08/09 21:17:20 INFO : Prompt attribute 'structured_examples' updated successfully.


In [66]:
new_model_parameter={"decoding_method":"sampling",
                 "max_new_tokens":204,
                 "min_new_tokens":1,
                 "random_seed":2,
                 "top_p":0
                }

In [67]:
structured_prompt.update_prompt_attribute(key_name="model_parameters",value=new_model_parameter)

2024/08/09 21:17:23 INFO : Prompt attribute 'model_parameters' updated successfully.


In [68]:
new_model_version={"number": "3.0.0", "tag": "tag1", "description": "Description of the new model version"}

In [69]:
structured_prompt.update_prompt_attribute(key_name="model_version",value=new_model_version)

2024/08/09 21:17:25 INFO : Prompt attribute 'model_version' updated successfully.


In [70]:
structured_prompt.get_info(verbose=True)

2024/08/09 21:17:26 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/ff222d89-69e3-4000-be83-a851e485a826/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:17:26 INFO : 
Prompt Details:
  id: ff222d89-69e3-4000-be83-a851e485a826
  name: structured-NewName
  input mode: structured
  model_id: ibm/granite-13b-chat-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e
  description: structured-description
  created By: IBMid-6910001S2E
  model_version: {'number': '3.0.0', 'tag': 'tag1', 'description': 'Description of the new model version'}
  prompt_variables: {'text': {'default_value': 'value'}, 'text_1': {'default_value': 'new_text1'}, 'user_1': {'default_value': 'new_user1'}}
  task_id: ['classification']
  instruction: new prompt_instruction
  input_prefix: new input_prefix
  output_prefix: new output_prefix
  structured_examples:
    input: What is the capital of India{text}?
    output: The 

#### Freeform prompts

In [71]:
freeform_prompt.update_prompt_attribute(key_name="task_id", value="classification")

2024/08/09 21:17:32 INFO : Prompt attribute 'task_id' updated successfully.


In [72]:
freeform_prompt.update_prompt_attribute(key_name="input", value="new input")

2024/08/09 21:17:35 INFO : Prompt attribute 'input' updated successfully.


In [73]:
new_model_parameter={"decoding_method":"sampling",
                 "max_new_tokens":204,
                 "min_new_tokens":1,
                 "random_seed":2,
                 "top_p":0
                }

In [74]:
freeform_prompt.update_prompt_attribute(key_name="model_parameters",value=new_model_parameter)

2024/08/09 21:17:37 INFO : Prompt attribute 'model_parameters' updated successfully.


In [75]:
freeform_prompt.get_info(verbose=True)

2024/08/09 21:17:38 INFO : The link to access the prompt created is: https://dataplatform.test.cloud.ibm.com/wx/prompt-details/f4ada477-9644-486e-9b07-22ca1584dd07/factsheet?project_id=544caa9f-82a5-4f5c-900e-db36097cc71e&context=wx
2024/08/09 21:17:38 INFO : 
Prompt Details:
  id: f4ada477-9644-486e-9b07-22ca1584dd07
  name: freeform-NewName
  input mode: freeform
  model_id: ibm/granite-13b-chat-v2
  container ID: 544caa9f-82a5-4f5c-900e-db36097cc71e
  description: freeform-description
  created By: IBMid-6910001S2E
  model_version: {}
  prompt_variables: {'text': {'default_value': 'value'}, 'text_1': {'default_value': 'new_text1'}, 'user_1': {'default_value': 'new_user1'}}
  task_id: ['classification']
  instruction: 
  input_prefix: 
  output_prefix: 
  structured_examples:
  input:

  model_parameters:
     decoding_method: sampling
     max_new_tokens: 204
     min_new_tokens: 1
     random_seed: 2
     stop_sequences: []
     temperature: 0.7
     top_k: 50
     top_p: 0
     re



---
## Creation of New AI Use Case <a href="#add_mut"></a>

An **AI Use Case** tracks model asset lifecycles across environments like development, pre-production, and production. 

**Note:** The term "AI Use Case" has replaced "Model Use Case" to reflect a broader range of AI assets. While some APIs may still use the old terminology, it will be phased out.

- If `ai_usecase_id` is not provided, the default inventory_id is used (requires `EDITOR` access).

- Retrieve the AI Use Case ID from the URL in inventory or by using `get_ai_usecase()`.

- For Cloud Pak for Data, ensure OpenPages integration is disabled (create inventory permission needed).


### Create AI usecase  <a name="createusecase"></a>

In [78]:
ai_usecase_inventory_id = os.getenv("INVENTORY_ID", "<if you can't use .env you can provide your inventory id here>")
ai_usecase_name="Prompt Scenarios Usecase"
ai_usecase_desc="AI usecase for Prompt Scenarios"

In [79]:
ai_usecase = facts_client.assets.create_ai_usecase(catalog_id=ai_usecase_inventory_id,name=ai_usecase_name,description=ai_usecase_desc)
ai_usecase.get_info(True)

2024/08/09 21:21:44 INFO : AI usecase created successfully


{'name': 'Prompt Scenarios Usecase',
 'description': 'AI usecase for Prompt Scenarios',
 'asset_type': 'model_entry',
 'url': 'https://dataplatform.cloud.ibm.com/data/catalogs/f8925e43-e661-4df7-8482-b6245433942c/asset/e96870bb-e6bc-4668-a44f-1614eedbefef?context=cpdaas',
 'model_usecase_id': 'e96870bb-e6bc-4668-a44f-1614eedbefef',
 'container_type': 'catalog',
 'catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'facts_type': 'model_entry_user'}

### Retrive AI usecase  <a name="createusecase"></a>

In [ ]:
ai_usecase_id = os.getenv("ai_usecase_id", "<if you can't use .env you can provide your inventory id here>")
ai_usecase_inventory_id = os.getenv("INVENTORY_ID", "<if you can't use .env you can provide your inventory id here>")



In [ ]:
ai_usecase = facts_client.assets.get_ai_usecase(ai_usecase_id=ai_usecase_id,catalog_id=ai_usecase_inventory_id )


In [ ]:
ai_usecase.get_info(True)  

In [80]:
print("Model usecase name is {}".format(ai_usecase.get_name()))
print("Model usecase ID is {}".format(ai_usecase.get_id()))

Model usecase name is Prompt Scenarios Usecase
Model usecase ID is e96870bb-e6bc-4668-a44f-1614eedbefef


---
## Associated Workspace

Associate workspacesis employed to associate AI use cases with the appropriate workspaces, ensuring they are organized around the same business problem. This involves setting up workspaces that will be used throughout the development process to tackle the business problem or implement the use case smoothly.


### Rules for Associating Workspaces

- A project can be associated with just one AI use case. Multiple projects might contribute assets or validations to a single AI use case, but each project is restricted to associating with a single-use case.

- A project can be associated with a single lifecycle phase for an AI use case. That is, you cannot use the same project to develop and validate an asset.

- A space can be associated with multiple AI use cases. That is, if you use a single space for deployed models in production, assets in that space might be tracked in different AI use cases. This distinction from how projects relate to an AI use case reflects how spaces are often used in ModelOps to manage a collection of deployments in a particular state.

`The following graphic illustrates the relationship of associated workspaces with an AI use case.`


<p align="center">
  <img src="https://dataplatform.cloud.ibm.com/docs/api/content/wsj/analyze-data/images/xgov-associate-workspace.svg?context=wx&locale=en" alt="Associate Workspace" style="width: 40%; max-width: 400px;">
</p>





<span style="color: gold; font-weight: bold;">**Note:** This feature is available in version 5.0.3 and above of the Cloud Pak for Data (CPD) release.</span>








### Retrieve All Associated Workspaces for the AI Use Case




In [81]:
ai_usecase.list_all_associated_workspaces()

model_usecase_id :e96870bb-e6bc-4668-a44f-1614eedbefef ,catalog_id : f8925e43-e661-4df7-8482-b6245433942c 
2024/08/09 21:22:50 INFO : Associated Workspaces retrieved successfully for the given AI usecase with id: e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:22:50 ERROR : No associated workspaces found for the specified model_usecase_id: e96870bb-e6bc-4668-a44f-1614eedbefef


#### Add Project Workspace to AI Use Case

This feature allows you to link a project workspace to an AI use case, ensuring that all related tasks, data, and documentation are organized together for efficient development and management.


In [82]:
project_id1=container_id

In [84]:
# add project workspace to model usecase
ai_usecase.add_workspaces_associations(workspace_id=project_id1, workspace_type="project", phase_name="develop")

model_usecase_id :e96870bb-e6bc-4668-a44f-1614eedbefef ,catalog_id : f8925e43-e661-4df7-8482-b6245433942c 
2024/08/09 21:24:06 INFO : Workspace associations attempted for the given workspace IDs: 544caa9f-82a5-4f5c-900e-db36097cc71e
2024/08/09 21:24:06 INFO : Workspace association for model use case with id e96870bb-e6bc-4668-a44f-1614eedbefef done successfully for the following workspace(s)==>

{'id': '544caa9f-82a5-4f5c-900e-db36097cc71e', 'type': 'project', 'name': 'task_cred_frankfurt', 'is_deleted': False}


#### Retrieve All Associated Workspaces for the Use Case by Phase



In [85]:
ai_usecase.list_associated_workspaces_by_phase(phase_name="validate")


model_usecase_id :e96870bb-e6bc-4668-a44f-1614eedbefef ,catalog_id : f8925e43-e661-4df7-8482-b6245433942c 
2024/08/09 21:24:36 INFO : Associated Workspaces retrieved successfully for the given AI usecase with id: e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:24:36 ERROR : No associated workspace found for the selected phase : validate


In [86]:
ai_usecase.list_associated_workspaces_by_phase(phase_name="develop")


model_usecase_id :e96870bb-e6bc-4668-a44f-1614eedbefef ,catalog_id : f8925e43-e661-4df7-8482-b6245433942c 
2024/08/09 21:24:37 INFO : Associated Workspaces retrieved successfully for the given AI usecase with id: e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:24:37 ERROR : No associated workspace found for the selected phase : develop


#### Remove Workspace Association

This feature allows you to unlink a project workspace from an AI use case, effectively removing the association. This action helps to manage workspace assignments and maintain accurate project configurations by disassociating workspaces that are no longer relevant or needed.


In [ ]:
ai_usecase.remove_workspace_associations(workspace_ids=[project_id2,space_id])


### Bulk Lookup of Workspaces

The **Bulk Lookup of Workspaces** feature allows you to efficiently retrieve information for multiple workspaces and projects in a single query. By specifying a list of identifiers and types, you can obtain details for various entities like projects and spaces, streamlining data retrieval and management.


In [87]:
workspace_info =[
                {"id": project_id1, "type": "project"}
                ]

facts_client.assets.get_workspace_association_info(workspace_info=workspace_info)


		====Workspace Details:====
* workspace id: 544caa9f-82a5-4f5c-900e-db36097cc71e
* workspace name: task_cred_frankfurt
* workspace type: project
* phase name: Develop
* is valid: True

		====Associated Use Cases:====
  * ai usecase id : e96870bb-e6bc-4668-a44f-1614eedbefef
  * ai usecase name: Prompt Scenarios Usecase
  * inventory id: f8925e43-e661-4df7-8482-b6245433942c
  * status: Draft
  * risk level: None

		====Tracked AI Assets:====

--------------------------------------------------



---
## Create an Approach <a href="#createapproach"></a>

- Track multiple prompt and prompts under a single use case by grouping them into different approaches.
- Create multiple approaches for various classification algorithms to facilitate comparison and integration.
- Use approaches to manage different prompt that need to be combined for a specific use case.


In [88]:
prompt_approach = ai_usecase.create_approach(name="Prompt Approach",description="Comprehensive Prompt Approach.")
prompt_approach.get_info()

2024/08/09 21:25:11 INFO : Approach created successfully


{'approach_id': '8af37f65-f98a-4baa-8c61-d6cbb26a4789',
 'approach_name': 'Prompt Approach',
 'approach_desc': 'Comprehensive Prompt Approach.',
 'model_asset_id': 'e96870bb-e6bc-4668-a44f-1614eedbefef',
 'model_container_type': 'catalog',
 'model_container_id': 'f8925e43-e661-4df7-8482-b6245433942c'}

---
## Track a Prompt Under an AI Use Case <a name="add_mu"></a>

- **AI Use Cases** are designed to monitor the lifecycle of prompt assets across various stages, including development, pre-production, and production.
- To effectively integrate a prompt into an AI use case, three critical elements must be addressed: the **prompt**, the **AI use case**, and the **approach**.
- Link an existing AI use case by using the following method: `prompt.track(usecase=<ai_usecase>, approach=<approach1>, version_number="<Version Number>")`.

- **Version Numbers** are categorized as follows:
    - **Major Version:** Indicates significant changes, represented as `1.0.0`.
    - **Minor Version:** Reflects incremental improvements, represented as `0.1.0`.
    - **Patch Version:** Denotes minor fixes or updates, represented as `0.0.1`.
    - **Custom Version:** Allows for tailored versioning according to specific user needs.

- Ensure that the `ai_usecase`, `approach`, and `version_number` parameters are provided as mandatory.


In [89]:
detached_prompt.track(usecase=ai_usecase,approach=prompt_approach,version_number="major")

-----------------------------------------------------------------------------------------------------------------------------
                                                   Tracking Process Started                                                  
-----------------------------------------------------------------------------------------------------------------------------
2024/08/09 21:25:15 INFO : Assigned Prompt Approach to Prompt Scenarios Usecase for tracking.
2024/08/09 21:25:16 INFO : Initiate linking model to existing AI usecase e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:25:21 INFO : Successfully finished linking Model 218352c9-600e-4ca9-8725-bb51c1a9bdf2 to AI usecase


{'model_entry_catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'model_entry_id': 'e96870bb-e6bc-4668-a44f-1614eedbefef',
 'model_entry_name': 'Prompt Scenarios Usecase',
 'model_entry_status': 'Draft',
 'version_number': '1.0.0',
 'version_comment': None,
 'approach_name': 'Prompt Approach'}

In [90]:
structured_prompt.track(usecase=ai_usecase,approach=prompt_approach,version_number="minor")

-----------------------------------------------------------------------------------------------------------------------------
                                                   Tracking Process Started                                                  
-----------------------------------------------------------------------------------------------------------------------------
2024/08/09 21:25:23 INFO : Assigned Prompt Approach to Prompt Scenarios Usecase for tracking.
2024/08/09 21:25:23 INFO : Initiate linking model to existing AI usecase e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:25:28 INFO : Successfully finished linking Model ff222d89-69e3-4000-be83-a851e485a826 to AI usecase


{'model_entry_catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'model_entry_id': 'e96870bb-e6bc-4668-a44f-1614eedbefef',
 'model_entry_name': 'Prompt Scenarios Usecase',
 'model_entry_status': 'Draft',
 'version_number': '1.1.0',
 'version_comment': None,
 'approach_name': 'Prompt Approach'}

In [91]:
freeform_prompt.track(usecase=ai_usecase,approach=prompt_approach,version_number="patch")

-----------------------------------------------------------------------------------------------------------------------------
                                                   Tracking Process Started                                                  
-----------------------------------------------------------------------------------------------------------------------------
2024/08/09 21:25:31 INFO : Assigned Prompt Approach to Prompt Scenarios Usecase for tracking.
2024/08/09 21:25:31 INFO : Initiate linking model to existing AI usecase e96870bb-e6bc-4668-a44f-1614eedbefef
2024/08/09 21:25:36 INFO : Successfully finished linking Model f4ada477-9644-486e-9b07-22ca1584dd07 to AI usecase


{'model_entry_catalog_id': 'f8925e43-e661-4df7-8482-b6245433942c',
 'model_entry_id': 'e96870bb-e6bc-4668-a44f-1614eedbefef',
 'model_entry_name': 'Prompt Scenarios Usecase',
 'model_entry_status': 'Draft',
 'version_number': '1.1.1',
 'version_comment': None,
 'approach_name': 'Prompt Approach'}

---
## Untrack a Prompt

Remove a Prompt from an AI use case when it is no longer relevant or needs to be managed separately.


In [92]:
detached_prompt.untrack()

2024/08/09 21:48:45 INFO : Successfully finished unregistering prompt 218352c9-600e-4ca9-8725-bb51c1a9bdf2 from AI use case.


In [93]:
structured_prompt.untrack()

2024/08/09 21:48:49 INFO : Successfully finished unregistering prompt ff222d89-69e3-4000-be83-a851e485a826 from AI use case.


In [94]:
freeform_prompt.untrack()

2024/08/09 21:48:53 INFO : Successfully finished unregistering prompt f4ada477-9644-486e-9b07-22ca1584dd07 from AI use case.


**Created by:**  


IBM watsonx.governance - *AI Factsheet Python SDK Team*

---

**Copyright © 2022-2024 IBM**  
Released under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0).
